In [1]:
# -----------------------------------------------------------------------------------------------------------------
# This jupyter notebook of ESC modeling construct approximate cell open circuit voltage (OCV) with dependence on 
# state of charge (SOC) and temperature.  It assumes that specific cell test scripts were run to generate the input
# data structure having fields of time, step, current, voltage, chgAh and disAh for each script run.
#
# Script 1 (thermal chamber set to test temperature):
# Step 1: Rest @ 100% SOC to acclimatize to test temperature 
# Step 2: Discharge @ low rate (ca. C/30) to min voltage 
# Step 3: Rest ca. 0%
# 
# Script 2 (thermal chamber set to 25 degC):
# Step 1: Rest ca. 0% SOC to acclimatize to 25 degC
# Step 2: Discharge to min voltage (ca. C/3)
# Step 3: Rest
# Step 4: Const voltage at vmin until current small (ca. C/30) 
# Steps 5-7: Dither around vmin
# Step 8: Rest
# Step 9: Constant voltage at vmin for 15 min
# Step 10: Rest
#
# Script 3 (thermal chamber set to test temperature): 
# Step 1: Rest at 0% SOC to acclimatize to test temp 
# Step 2: Charge @ low rate (ca. C/30) to max voltage 
# Step 3: Rest
#
# Script 4 (thermal chamber set to 25 degC):
# Step 1: Rest ca. 100% SOC to acclimatize to 25 degC
# Step 2: Charge to max voltage (ca. C/3)
# Step 3: Rest
# Step 4: Const voltage at vmax until current small (ca. C/30) 
# Steps 5-7: Dither around vmax
# Step 8: Rest
# Step 9: Constant voltage at vmax for 15 min
# Step 10: Rest
# -----------------------------------------------------------------------------------------------------------------

In [2]:
import pandas as pd
import numpy as np
from scipy import interpolate
from scipy.linalg import lstsq
import pickle
from ESC_modeling import ESCmodel

In [3]:
# -----------------------------------------------------------------------------------------------------------------
# Specify cell and test parameters
# -----------------------------------------------------------------------------------------------------------------

cellID = 'P14'
minV = 2.50  # the lower bound of voltage contrain
maxV = 4.25  # the upper bound of voltage contrain 
temps = [-25,-15,-5,5,15,25,35,45]  # temperatures at which cell tests were conducted

In [4]:
# -----------------------------------------------------------------------------------------------------------------
# Initaite a model
# -----------------------------------------------------------------------------------------------------------------

Model = ESCmodel()
Model.SOC = np.arange(0.0,1.0+0.005,0.005)
Model.OCV = np.arange(minV-0.01,maxV+0.02,0.01)

eta = np.zeros(len(temps)) 
Q = np.zeros(len(temps)) 

In [5]:
# -----------------------------------------------------------------------------------------------------------------
# Load raw data of cell test
# -----------------------------------------------------------------------------------------------------------------

OCVdir = cellID + '_OCV'
OCVdata = {}

for T in temps:
    OCVdata_temp = {}
    
    for script in [1,2,3,4]:
        if T > 0:
            OCVfile = '%s_P%02d_S%d.xlsx' % (OCVdir,T,script)
        else:
            OCVfile = '%s_N%02d_S%d.xlsx' % (OCVdir,abs(T),script)
        
        data = pd.read_excel(OCVdir+'/'+OCVfile,'Channel_1-003')
        data = data[['Test_Time(s)','Step_Index','Current(A)','Voltage(V)',\
                     'Charge_Capacity(Ah)','Discharge_Capacity(Ah)']]
        data.columns = ['time','step','current','voltage','chgAh','disAh']
        
        OCVdata_temp[str(script)] = data
        
    OCVdata[str(T)] = OCVdata_temp

In [6]:
# -----------------------------------------------------------------------------------------------------------------
# Find coulombic efficiency and capacity at 25 degC
# -----------------------------------------------------------------------------------------------------------------

totDisAh = 0
totChgAh = 0

for script in [1,2,3,4]:
    totDisAh = totDisAh + OCVdata[str(25)][str(script)]['disAh'].iloc[-1]
    totChgAh = totChgAh + OCVdata[str(25)][str(script)]['chgAh'].iloc[-1]
eta25 = totDisAh / totChgAh
eta[temps.index(25)] = eta25

for script in [1,2,3,4]:
    OCVdata[str(25)][str(script)]['chgAh'] = OCVdata[str(25)][str(script)]['chgAh']*eta25

Q25 = OCVdata[str(25)]['1']['disAh'].iloc[-1] + OCVdata[str(25)]['2']['disAh'].iloc[-1]\
    - OCVdata[str(25)]['1']['chgAh'].iloc[-1] - OCVdata[str(25)]['2']['chgAh'].iloc[-1]
Q[temps.index(25)] = Q25


# -----------------------------------------------------------------------------------------------------------------
# Find coulombic efficiency and capacity at other temperatures
# -----------------------------------------------------------------------------------------------------------------

for x,T in enumerate(temps):
    if T != 25:
        OCVdata[str(T)]['2']['chgAh'] = OCVdata[str(T)]['2']['chgAh']*eta25
        OCVdata[str(T)]['4']['chgAh'] = OCVdata[str(T)]['4']['chgAh']*eta25
       
        etaT = (OCVdata[str(T)]['1']['disAh'].iloc[-1]\
                +OCVdata[str(T)]['2']['disAh'].iloc[-1]\
                +OCVdata[str(T)]['3']['disAh'].iloc[-1]\
                +OCVdata[str(T)]['4']['disAh'].iloc[-1]\
                -OCVdata[str(T)]['2']['chgAh'].iloc[-1]\
                -OCVdata[str(T)]['4']['chgAh'].iloc[-1])\
                /(OCVdata[str(T)]['1']['chgAh'].iloc[-1]\
                +OCVdata[str(T)]['3']['chgAh'].iloc[-1])
        
        OCVdata[str(T)]['1']['chgAh'] = OCVdata[str(T)]['1']['chgAh']*etaT
        OCVdata[str(T)]['3']['chgAh'] = OCVdata[str(T)]['3']['chgAh']*etaT
        eta[x] = etaT
        
        Q[x] = OCVdata[str(T)]['1']['disAh'].iloc[-1] + OCVdata[str(T)]['2']['disAh'].iloc[-1]\
            - OCVdata[str(T)]['1']['chgAh'].iloc[-1] - OCVdata[str(T)]['2']['chgAh'].iloc[-1]

In [7]:
# -----------------------------------------------------------------------------------------------------------------
# Find approximate OCV-SOC relationship at each temp
# -----------------------------------------------------------------------------------------------------------------

rawOCV = np.zeros((len(temps),len(Model.SOC)))

for x,T in enumerate(temps):

    #Compute ohmic resistance (R0) estimates
    indD = OCVdata[str(T)]['1'].index[OCVdata[str(T)]['1']['step']==2].tolist()
    IR1Da = OCVdata[str(T)]['1']['voltage'][indD[0]-1] - OCVdata[str(T)]['1']['voltage'][indD[0]]
    IR2Da = OCVdata[str(T)]['1']['voltage'][indD[-1]+1] - OCVdata[str(T)]['1']['voltage'][indD[-1]]

    indC = OCVdata[str(T)]['3'].index[OCVdata[str(T)]['3']['step']==2].tolist()
    IR1Ca = OCVdata[str(T)]['3']['voltage'][indC[0]] - OCVdata[str(T)]['3']['voltage'][indC[0]-1]
    IR2Ca = OCVdata[str(T)]['3']['voltage'][indC[-1]] - OCVdata[str(T)]['3']['voltage'][indC[-1]+1]

    IR1D = min(IR1Da, 2*IR2Ca)
    IR2D = min(IR2Da, 2*IR1Ca)
    IR1C = min(IR1Ca, 2*IR2Da)
    IR2C = min(IR2Ca, 2*IR1Da)

    # Adjust voltage curves: compensate dis/charge curves for R0*i(t)
    IRDblend = [IR1D + (IR2D-IR1D)*x/(len(indD)-1) for x in range(len(indD))]
    disV = OCVdata[str(T)]['1']['voltage'][indD[:]] + IRDblend
    disZ = 1 - OCVdata[str(T)]['1']['disAh'][indD[:]]/Q25
    disZ = disZ + (1-disZ.iloc[0])  #force initial 100% SOC

    IRCblend = [IR1C + (IR2C-IR1C)*x/(len(indC)-1) for x in range(len(indC))]
    chgV = OCVdata[str(T)]['3']['voltage'][indC[:]] - IRCblend
    chgZ = OCVdata[str(T)]['3']['chgAh'][indC[:]]/Q25
    chgZ = chgZ - chgZ.iloc[0]  #force initial 0% SOC
    
    # Compensate for steady-state resistance
    deltaV50 = interpolate.interp1d(chgZ,chgV)(0.5) - interpolate.interp1d(disZ,disV)(0.5)

    cInd = chgV.index[chgZ<0.5].tolist()
    zChg = chgZ[cInd[:]]
    vChg = chgV[cInd[:]] - zChg*deltaV50

    dInd = disV.index[disZ>0.5].tolist()
    zDis = disZ[dInd[:]]
    vDis = disV[dInd[:]] + (1-zDis)*deltaV50
    
    # Compute OCV corresponding to specified SOC
    rawocv = interpolate.interp1d(pd.concat([zDis,zChg]),pd.concat([vDis,vChg]))(Model.SOC)
    
    rawOCV[x] = rawocv

In [8]:
# -----------------------------------------------------------------------------------------------------------------
# Combine multiple OCV-SOC relationships into a temepature-depedent OCV-SOC relationship: 
# OCV(z,T) = OCV0(z) + T*OCVrel(z)
# -----------------------------------------------------------------------------------------------------------------

# The approximate OCV relationships obtained below 0 degC are discarded becuase they severely deviate from true OCV
# due to much higher resistance and hysteresis
A = np.array([[1,T] for T in temps if T>0])
Y = rawOCV[[x for x,T in enumerate(temps) if T>0],:]
X = np.linalg.lstsq(A, Y)

# store the OCV-SOC relationship in the model
Model.OCV0 = X[0][0]
Model.OCVrel = X[0][1]

In [9]:
# -----------------------------------------------------------------------------------------------------------------
# Find temepature-depedent SOC-OCV relationship: SOC(v,T) = SOC0(v) + T*SOCrel(v)
# -----------------------------------------------------------------------------------------------------------------

rawSOC = np.zeros((len(temps),len(Model.OCV)))
z = np.arange(-0.1,1.1+0.01,0.01)

for x,T in enumerate(temps):
    v = Model.OCVfromSOCtemp(z,T)
    rawSOC[x] = interpolate.interp1d(v,z)(Model.OCV)

A = np.array([[1,T] for T in temps])
Y = rawSOC
X = np.linalg.lstsq(A, Y)

# store the SOC-OCV relationship in the model
Model.SOC0 = X[0][0]
Model.SOCrel = X[0][1]

In [10]:
# -----------------------------------------------------------------------------------------------------------------
# Export the model object
# -----------------------------------------------------------------------------------------------------------------

model_file = cellID + '_model.pickle'
pickle.dump(Model, open(model_file, 'wb'))